# The Self-Conscious Statisticians

Imagine a group of statisticians who are in a weight loss program.  Because they're statisticians, they want to compute summary statistics over the group weights.  Because they're self-conscious, none want to reveal their weight to the group.  This is a perfect use-case for Cicada and MPC:

In [1]:
import logging

import numpy

import cicada.additive
import cicada.logging
from cicada.communicator import SocketCommunicator

logging.basicConfig(level=logging.INFO)

def main(communicator):
    log = cicada.logging.Logger(logging.getLogger(), communicator)
    protocol = cicada.additive.AdditiveProtocol(communicator)

    # Each player loads their weight from a file.
    weight = numpy.loadtxt(f"statistician-weight-{communicator.rank}.txt")
        
    # Compute the sum of the player weights.
    mean_share = protocol.share(src=0, secret=protocol.encoder.zeros(shape=()), shape=())
    for rank in communicator.ranks:
        weight_share = protocol.share(src=rank, secret=protocol.encoder.encode(weight), shape=weight.shape)
        mean_share = protocol.add(mean_share, weight_share)
        
    # Divide by the number of players to obtain the mean weight.
    mean_share = protocol.untruncated_private_public_divide(mean_share, communicator.world_size)
    mean_share = protocol.truncate(mean_share)
                                 
    # Reveal the mean weight to the group.
    mean = protocol.encoder.decode(protocol.reveal(mean_share))
                                 
    log.info(f"Mean weight revealed to player {communicator.rank}: {mean}")
    
SocketCommunicator.run(world_size=5, fn=main);

INFO:cicada.communicator.socket:Comm 'world' player 0 rendezvous with tcp://127.0.0.1:49260 from tcp://127.0.0.1:49260.
INFO:cicada.communicator.socket:Comm 'world' player 1 rendezvous with tcp://127.0.0.1:49260 from tcp://127.0.0.1:49261.
INFO:cicada.communicator.socket:Comm 'world' player 2 rendezvous with tcp://127.0.0.1:49260 from tcp://127.0.0.1:49262.
INFO:cicada.communicator.socket:Comm 'world' player 3 rendezvous with tcp://127.0.0.1:49260 from tcp://127.0.0.1:49266.
INFO:cicada.communicator.socket:Comm 'world' player 4 rendezvous with tcp://127.0.0.1:49260 from tcp://127.0.0.1:49272.
INFO:cicada.communicator.socket:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 4 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 2 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 3 communicator ready.
INFO:root:Mean weight revealed

If we manually inspect the players' weights, we can see that the result is correct, making allowances for the default 16-bit fixed point precision:

In [2]:
weights = [numpy.loadtxt(f"statistician-weight-{rank}.txt") for rank in range(5)]
print(f"weights: {weights}")
print(f"mean: {numpy.mean(weights)}")

weights: [array(130.), array(220.), array(98.), array(241.), array(215.)]
mean: 180.8
